In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare  HELICOID()

In [3]:
function helicoid(R=1., r=0.5, pitch=1., nturns=2)
    function helicoid0(shape=[36*nturns, 2])
        angle = nturns*2*pi
        V, CV = Lar.simplexGrid(shape)
        V = [angle/shape[1] 0;0 (R-r)/shape[2]]*V
        V = broadcast(+, V, [0, r])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u);
        (pitch/(2*pi))*u] end, W)...)
        return V, CV
    end
    return helicoid0
end


helicoid (generic function with 5 methods)

In [4]:
@btime W,CW = Lar.helicoid()();

  207.998 μs (3155 allocations: 379.17 KiB)


In [5]:
@code_warntype Lar.helicoid()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#helicoid0#274"{Float64,Float64,Float64,Int64}

Body::Tuple{Any,Any}
1 ─ %1 = Core.getfield(#self#, :nturns)::Int64
│   %2 = (36 * %1)::Int64
│   %3 = Base.vect(%2, 2)::Array{Int64,1}
│   %4 = (#self#)(%3)::Tuple{Any,Any}
└──      return %4


In [6]:
@benchmark Lar.helicoid()()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  217.511 μs …   7.523 ms  ┊ GC (min … max):  0.00% … 90.99%
 Time  (median):     385.938 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   408.129 μs ± 518.553 μs  ┊ GC (mean ± σ):  10.99% ±  8.26%

   ▆█▂                     ▁                                     
  ▄███▆▄▃▃▃▃▂▃▂▃▃▃▃▄▅▅▆▆▇█████▇▆▅▄▄▃▃▃▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  218 μs           Histogram: frequency by time          675 μs <

 Memory estimate: 379.11 KiB, allocs estimate: 3153.

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [16]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [21]:
function helicoid2(R=1., r=0.5, pitch=1., nturns=2)
    function helicoid02(shape=[36*nturns, 2])
        angle = nturns*2*pi
        V, CV = Lar.simplexGrid(shape)
        V = [angle/shape[1] 0;0 (R-r)/shape[2]]*V
        V = broadcast(+, V, [0, r])
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
        end 
        V = hcat(map(p->let(u, v)=p;[v*cos(u);v*sin(u);
        (pitch/(2*pi))*u] end, W)...)
        return V, CV
    end
    return helicoid02
end

helicoid2 (generic function with 5 methods)

In [22]:
@btime W,CW = helicoid2()();

  186.293 μs (2817 allocations: 328.83 KiB)


E' sorprendente vedere che helicoid è una delle poche funzioni che non utilizza simplifyCells , con l'applicazione delle macro settando nel giusto modo i threads , le prestazioni sono leggermente migliorate come si puo vedere.   